<a href="https://colab.research.google.com/github/blockchainrelativity/Colab_Experiments/blob/main/meeting_transcripts_w_speaker_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notes on usage:

- Make sure to [change runtime to GPU](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- The transcript will be saved in Files, which you can find in the menu on the left.
- Change the number of speakers below if different from two.
- Pick a bigger model if you want more accuracy and a smaller model if you want the program to run faster ([more info](https://github.com/openai/whisper#available-models-and-languages)).
- If you know the language being spoken is English, then change language to 'English' as this improves performance.


High level overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.   

Let me know if I can make it better!


In [ ]:
# BE SURE TO SET THESE VARIABLES in the appropriate cell:

# input_file_path
# num_speakers =  #@param {type:"integer"}
# language = '' #@param ['any', 'English']
# model_size

In [ ]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# *****Set chosen file path here******
input_file_path = "/content/drive/MyDrive/Meet Recordings/Emurgo Hackathon - Moving Parts (2023-05-21 15:05 GMT-5) (1)"

In [ ]:
# Creating directory for export of audio files into wav format
!mkdir transcripts

In [ ]:
# Use when source is MP4. Convert MP4 into wav for model

!pip install moviepy

from moviepy.editor import AudioFileClip

# Set the paths of the input and output files
# input_file_path = "audio.mp4"
output_file_path = "transcripts/audio.wav"

# Load the video file
video = AudioFileClip(input_file_path)

# Extract the audio and save it as a WAV file
audio = video.write_audiofile(output_file_path, codec='pcm_s16le')
# , ffmpeg_params=['-num_channels','1']


In [ ]:
# # Use when source is m4a

# !pip install pydub
# from pydub import AudioSegment

# # Set the input and output file paths
# input_file = "audio1039974750.m4a"
# output_file = "transcripts/audio.wav"

# # Read the input file
# audio = AudioSegment.from_file(input_file, format="m4a")

# # Export the audio as a WAV file
# audio.export(output_file, format="wav")


In [ ]:
num_speakers = 5 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'medium' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'medium':
  model_name += '.en'


In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
# # ***Alternative to moviepy***
# # pip install cloudconvert. This will convert many file types.

# import cloudconvert

# cloudconvert.configure(api_key = 'API_KEY', sandbox = False)

# import CloudConvert from 'cloudconvert';

# const cloudConvert = new CloudConvert('api_key');

# let job = await cloudConvert.jobs.create({
#     "tasks": {
#         "import-mp4": {
#             "operation": "import/upload"
#         },
#         "task-1": {
#             "operation": "convert",
#             "input_format": "mp4",
#             "output_format": "wav",
#             "engine": "ffmpeg",
#             "input": [
#                 "import-mp4"
#             ],
#             "audio_codec": "pcm_s16le",
#             "audio_bitrate": 128,
#             "engine_version": "5.1.2"
#         }
#     },
#     "tag": "jobbuilder"
# });

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))
# Model options: dwarkesh/whisper-speaker-recognition, speechbrain/spkrec-ecapa-voxceleb

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
# import m4a
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
# if path[-3:] != 'wav':
#   subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
#   path = 'audio.wav'

path = '/content/transcripts/audio.wav'

In [ ]:
model = whisper.load_model(model_size)

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [ ]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [ ]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [ ]:
!pip install pyannote.audio

In [ ]:
# from pyannote.audio. import SequenceEmbedding
!pip install librosa

# Load the audio data and convert it to mono
audio, sample_rate = librosa.load(path, sr=None, mono=True)

# Initialize the embedding extractor
embedding_extractor = SequenceEmbedding()

# Extract the embeddings for each segment of the audio
segments = [audio[start:end] for start, end in segment_times]
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = embedding_extractor(segment)

# Replace any NaN values with 0
embeddings = np.nan_to_num(embeddings)


In [ ]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [ ]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [ ]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

# Use Transcription to get Summary

In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install "unstructured[local-inference]"
!pip install Pillow
!pip install tiktoken

In [ ]:
#  If using an external .txt file not processed by whisper, run this cell
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

In [ ]:
from langchain.chains.summarize import load_summarize_chain
# from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain import OpenAI, PromptTemplate
import glob
from langchain.llms.openai import OpenAI
# from langchain.utils import truncate_text

# import glob
# from openai import OpenAI, UnstructuredFileLoader, PromptTemplate, load_summarize_chain
# from openai.util import truncate_text

In [ ]:
# llm = OpenAI(temperature=0.2)
# def summarize_pdfs_from_folder(pdfs_folder):
#     summaries = []
#     for pdf_file in glob.glob(pdfs_folder + "/*.pdf"):
#         loader = PyPDFLoader(pdf_file)
#         docs = loader.load_and_split()
#         chain = load_summarize_chain(llm, chain_type="map_reduce")
#         summary = chain.run(docs)
#         print("Summary for: ", pdf_file)
#         print(summary)
#         print("\n")
#         summaries.append(summary)

#     return summaries

In [ ]:
OPENAI_API_KEY = 'sk-'
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0)

text_splitter = CharacterTextSplitter()

In [ ]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

text_splitter = CharacterTextSplitter()
transcription_file = "./transcripts/transcript.txt"
llm = OpenAI(temperature=0, verbose=True)

with open(transcription_file, "r") as f:
        text = f.read()
texts = text_splitter.split_text(text)

from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:3]]

CUSTOM_PROMPT = """Write a concise summary of the following meeting transcription. Our goals in these meetings are to Identify strategic priorities and define specific tactics and initiatives. Additionally include any information or insight related to those goals in the summary with this structure:
Problem being solved;
Approach;
Main results;
Main Discussion Points;
List of Topics Discussed;
Full concise summary;  """

prompt_template = CUSTOM_PROMPT + """

{text}

SUMMARY:"""



PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
summary_output = chain.run(docs)

with open("custom_summary.txt", "w") as f:
    f.write(summary_output)

In [ ]:
# Run this if using a single text file that contains the transcription


transcription_file = "./transcripts/transcript.txt"
llm = OpenAI(temperature=0.4, verbose=True)

def custom_summary(transcription_file, custom_prompt):
    with open(transcription_file, "r") as f:
        text = f.read()
    prompt_template = CUSTOM_PROMPT + """

    {text}

    SUMMARY:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
    chain.run(docs)
    return chain

CUSTOM_PROMPT = """Write a concise summary of the following meeting transcription. Our goals in these meetings are to Identify strategic priorities and define specific tactics and initiatives. Additionally include any information or insight related to those goals in the summary with this structure:
Problem being solved;
Approach;
Main results;
Main Discussion Points;
List of Topics Discussed;
Full concise summary;  """

custom_summary = custom_summary(transcription_file, custom_prompt=CUSTOM_PROMPT)

# Save the summary into a .txt file
with open("custom_summary.txt", "w") as f:
    f.write(custom_summary)



In [ ]:
# transcriptions = "./transcripts/"

# llm = OpenAI(temperature=0.4, verbose=True)

# def custom_summary(transcriptions, custom_prompt, max_tokens=700):
#     summaries = []
#     for txt_file in glob.glob(transcriptions + "*.txt"):
#         with open(txt_file, "r") as f:
#             text = f.read()
#         prompt_template = custom_prompt + """

#         {text}

#         SUMMARY:"""
#         PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
#         loader = UnstructuredFileLoader(txt_file)
#         docs = loader.load_and_split()
#         chain = load_summarize_chain(llm, chain_type="map_reduce",
#                                     map_prompt=PROMPT, combine_prompt=PROMPT)
#         summary_output = chain({"input_documents": docs}, return_only_outputs=True)["output_text"]
#         summary_output = truncate_text(summary_output, max_tokens)
#         summaries.append(summary_output)

#     return summaries

# CUSTOM_PROMPT = """Write a concise summary of the following meeting transcriptions. Our goals in these meetings are to Identify strategic priorities and define specific tactics and initiatives. Additionally include any information or insight related to those goals in the summary with this structure:
# Problem being solved;
# Approach;
# Main results;
# Main Discussion Points;
# List of Topics Discussed;
# Full concise summary;  """

# custom_summaries = custom_summary(transcriptions, custom_prompt=CUSTOM_PROMPT)

# # Save all summaries into one .txt file
# with open("custom_summaries.txt", "w") as f:
#     for summary in custom_summaries:
#         f.write(summary + "\n"*3)

In [ ]:
# # Run this if using the audio file instead of a precompiled transcription text file
# llm = OpenAI(temperature=0.2, verbose=True)

# def custom_summary(transcriptions, custom_prompt):
#     summaries = []
#     for txt_file in glob.glob(transcriptions + "/*.txt"):
#         loader = UnstructuredFileLoader(txt_file)
#         docs = loader.load_and_split()
#         prompt_template = custom_prompt + """

#         {text}

#         SUMMARY:"""
#         PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
#         chain = load_summarize_chain(llm, chain_type="map_reduce",
#                                     map_prompt=PROMPT, combine_prompt=PROMPT)
#         summary_output = chain({"input_documents": docs}, return_only_outputs=True)["output_text"]
#         summaries.append(summary_output)

#     return summaries


# CUSTOM_PROMPT = "Write a concise summary of the following meeting transcriptions with this structure: Problem being solved; Approach; Main results; Main Discussion Points; List of Topics Discussed.  Our goals in these meetings are to Identify strategic priorities and define specific tactics and initiatives. Include any information or insight related to those goals in the summary"
# custom_summaries = custom_summary("./txt", custom_prompt=CUSTOM_PROMPT)
# # Save all summaries into one .txt file
# with open("custom_summaries.txt", "w") as f:
#     for summary in custom_summaries:
#         f.write(summary + "\n"*3)

In [ ]:
# Run this if using multiple PDFs

# summaries = summarize_pdfs_from_folder("./pdfs")

# # Save all summaries into one .txt file
# with open("summaries.txt", "w") as f:
#     for summary in summaries:
#         f.write(summary + "\n"*3)

# BREAK - This is a test

In [ ]:
!pip install gradio

import whisper
import gradio as gr
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

model = whisper.load_model("large-v2")
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

def transcribe(audio, num_speakers):
  path, error = convert_to_wav(audio)
  if error is not None:
    return error

  duration = get_duration(path)
  if duration > 4 * 60 * 60:
    return "Audio duration too long"

  result = model.transcribe(path)
  segments = result["segments"]

  num_speakers = min(max(round(num_speakers), 1), len(segments))
  if len(segments) == 1:
    segments[0]['speaker'] = 'SPEAKER 1'
  else:
    embeddings = make_embeddings(path, segments, duration)
    add_speaker_labels(segments, embeddings, num_speakers)
  output = get_output(segments)
  return output

def convert_to_wav(path):
  if path[-3:] != 'wav':
    new_path = '.'.join(path.split('.')[:-1]) + '.wav'
    try:
      subprocess.call(['ffmpeg', '-i', path, new_path, '-y'])
    except:
      return path, 'Error: Could not convert file to .wav'
    path = new_path
  return path, None

def get_duration(path):
  with contextlib.closing(wave.open(path,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    return frames / float(rate)

def make_embeddings(path, segments, duration):
  embeddings = np.zeros(shape=(len(segments), 192))
  for i, segment in enumerate(segments):
    embeddings[i] = segment_embedding(path, segment, duration)
  return np.nan_to_num(embeddings)

audio = Audio()

def segment_embedding(path, segment, duration):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

def add_speaker_labels(segments, embeddings, num_speakers):
  clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
  labels = clustering.labels_
  for i in range(len(segments)):
    segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

def time(secs):
  return datetime.timedelta(seconds=round(secs))

def get_output(segments):
  output = ''
  for (i, segment) in enumerate(segments):
    if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
      if i != 0:
        output += '\n\n'
      output += segment["speaker"] + ' ' + str(time(segment["start"])) + '\n\n'
    output += segment["text"][1:] + ' '
  return output

gr.Interface(
    title = 'Whisper with Speaker Recognition',
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="upload", type="filepath"),
        gr.inputs.Number(default=2, label="Number of Speakers")

    ],
    outputs=[
        gr.outputs.Textbox(label='Transcript')
    ]
  ).launch()